In [6]:
from IPython.display import clear_output
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler # проверить все

In [7]:
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
%matplotlib inline

import os, time
import warnings
warnings.filterwarnings("ignore")

In [8]:
analyze_period = 10000    # Сколько баров в прошлое берем для тренировки модели
ensemble = 5
forvard_lag = 0

In [13]:
path_data = 'Palm-oil prices forecasting/data/'
path_models = 'Palm-oil prices forecasting/models/'
file_X = 'dataframe_1.csv'
file_Y = 'dataframe_Y.csv'
#os.chdir(path_data)
print(os.listdir('Palm-oil prices forecasting/models/'))

FileNotFoundError: [Errno 2] No such file or directory: 'Palm-oil prices forecasting/models/'

In [ ]:
# Получим список факторов, которые нужно удалить из Х
os.chdir(path_data)
with open('spisok_factors_for_del.txt', 'r') as f:
    spisok_factors_for_del = f.read()
spisok_factors_for_del = spisok_factors_for_del.split("\n")
spisok_factors_for_del

In [ ]:
os.chdir(path_models)
models_list = os.listdir()
models_list

### Dataset loading

In [ ]:
os.chdir(path_data)

In [ ]:
#Загрузка данных
data_X = pd.read_csv(file_X, sep=",", index_col=['date'])
data_X = data_X[-analyze_period:] # Убрали устаревшую информацию
print(data_X.shape)
data_X[-3:]

In [ ]:
del data_X['Пальмовое масло   RBD, FOB Малайзия , usd/тн ']
for i in spisok_factors_for_del: del data_X[i]
data_X[-3:]

In [ ]:
# Загрузка Y
data_Y = pd.read_csv(file_Y, sep=",", index_col=['date'])
data_Y.rename(columns={'Пальмовое масло   RBD, FOB Малайзия , usd/тн ': 'Y'}, inplace=True)
data_Y = data_Y[-analyze_period:] # Убрали устаревшую информацию
print(data_Y.shape)
data_Y[-3:]

### Функции разделения массива на ансамбли

In [ ]:
"""
#### Функция разделения массива на выборки для обучения нейросети 
"""
# sequence = строка массива df
# trend = число баров в прошлом для анализа - необходимо найти оптимальный!!
# forvard_lag = предсказание какого бара в будущем делаем 

def split_sequence(sequence, ensemble, forvard_lag=0):
  # sequence = df.values - это массив из данных
  # trend = число баров в прошлом для анализа
  X = list()
  for i in range(len(sequence)):
    end_ix = i + ensemble # находим конечный индекс строки
    if end_ix + (forvard_lag-1) > len(sequence)-1: # cпроверем чтобы не выйти за пределы массива
      break 
    # получаем X и y
    seq_x = sequence[i:end_ix]
    X.append(seq_x)

  return array(X)

## Prediction

### Подготовка данных для подачи в модель для предикта

In [ ]:
xTrain = data_X.values
print('xTrain.shape: ', xTrain.shape)

In [ ]:
xScaler = StandardScaler()
#xScaler = RobustScaler()
#xScaler = MinMaxScaler()
xScaler.fit(xTrain)
xTrain = xScaler.transform(xTrain)
print('xTrain[-1]: ', xTrain[-1])
print('xScaler.inverse_transform(xTrain[-1:]):', xScaler.inverse_transform(xTrain[-1:]))

In [5]:
# Формируем выборки для обучения нейросети
xTrain = split_sequence(xTrain, ensemble)
print(xTrain.shape)
print('xTrain[-1]:\n', xTrain[-1])
print('xScaler.inverse_transform(xTrain[-1]):\n', xScaler.inverse_transform(xTrain[-1]))

NameError: name 'split_sequence' is not defined

### Предикт по трем сохраненным моделям и отрисовка графика

In [ ]:
df_pred = pd.DataFrame()

for model_name in models_list:
  # Загружаем модель и расшифровываем настройки ансамблей
  #ensemble = int(model_name[12:14])
# ---------------------------------------------------------------------------

  file = model_name
  os.chdir(path_models)
  model = load_model(file)

  predictions = model.predict(xTrain)
  #print(model_name, predictions.shape, '\n', predictions[-3:], '\n')

  # Преоразования дляг рафика
  predictions[:-30] = np.NaN
  predictions = predictions.reshape(predictions.shape[0])
  df_pred[model_name[:-3]] = predictions

#df_pred
# Добавляем в датасет Y_fact
# Удалим из датафрейма Y начало из 34 элементов
data_Y_modify = data_Y[34:]   #берем последние 2618 элементов
#Запишем колонку date в другой датафрейм, а Y оставим без него
index_data_Y = pd.DataFrame(data_Y_modify.index)
Y_1 = data_Y_modify.values
#Теперь допишем нулями 30 значений вперед
for x in range(30):
    Y_1 = np.append(Y_1, [np.nan])

#Запишем данные Y в основной датафрейм предсказания
df_pred['Y_fact'] = Y_1
#df_pred[-35:]

# ---------------- график --------------------

col_names = ','.join(df_pred.columns)
col_names = col_names.split(',')

fig,axes = plt.subplots(nrows=1, ncols=1, figsize=(20,6)) 
#  Методы, отображающие данные:
axes.plot(df_pred[-360:]) # вывведем последний год цен на масло
axes.legend(col_names,fontsize=24)
axes.grid()
#ax.legend(["AAA", "BBB"]);
plt.show()